In [44]:
!pip install python-sat

In [7]:
import random
import numpy as np
from pysat.solvers import Glucose4

def gerar_instancia(n, m, k):

   # Gera instâncias de k-SAT sem repetições de cláusulas ou literais contraditórios.

    clausulas = []
    clausulas_unicas = set()

    while len(clausulas) < m:
        clausula = set()
        while len(clausula) < k:
            valor = random.randint(1, n)
            sinal = random.choice([1, -1]) #distribui aleatóriamente os sinais dos literais
            literal = valor * sinal

            # Evita que sejam adicionados os mesmo literais ou literais contraditorios
            if literal not in clausula and -literal not in clausula:
                clausula.add(literal)

        # Garante clausulas únicas
        clausula_congelada = frozenset(clausula)
        if clausula_congelada not in clausulas_unicas:
            clausulas_unicas.add(clausula_congelada)
            clausulas.append(list(clausula))

    return clausulas

In [26]:
def executar_experimento():
    resultados = {}
    n_valores = [50, 100, 150, 200]
    n_valores_5_sat=[20, 30, 40, 50]
    k_valores = [3, 5]


    for k in k_valores:
        resultados[k] = {}
        if k ==5:
          n_valores = n_valores_5_sat
        else:
          n_valores=n_valores


        alfa_min, alfa_max, aumento = (1.0, 10.0, 0.1) #estabelece a variação de alpha no k=3
        if k == 5:
          alfa_min, alfa_max, aumento = (1.0, 30.0, 0.1)
        alfas = np.round(np.arange(alfa_min, alfa_max + aumento, aumento), 1) #estabelece a variação de alpha no k=5


        for n in n_valores:
            resultados[k][n] = {'alfas': [], 'probabilidades': [], 'tempos': []}
            print(f"\nProcessando {k}-SAT com n={n}...")


            for alfa in alfas:
                m = int(alfa * n)
                if m < 1:
                  m = 1  # começa com no mínimo 1 cláusula


                sat_count = 0
                tempo_total = 0
                instancias = 30


                for _ in range(instancias):  # 30 instâncias por alfa
                    clausulas = gerar_instancia(n, m, k)
                    solver = Glucose4(use_timer=True)


                    for clausula in clausulas:
                        solver.add_clause(clausula)


                    is_sat = solver.solve()
                    tempo_total += solver.time()
                    sat_count += 1 if is_sat else 0
                    solver.delete()


                probabilidade = sat_count / instancias
                tempo_medio = tempo_total / instancias


                resultados[k][n]['alfas'].append(alfa)
                resultados[k][n]['probabilidades'].append(probabilidade)
                resultados[k][n]['tempos'].append(tempo_medio)


                print(f"α={alfa:.1f} | Prob: {probabilidade:.2f} | Tempo: {tempo_medio:.4f}s")


    return resultados


In [29]:
import matplotlib.pyplot as plt
def gerar_graficos(resultados):
    # Grafico de probabilidade
    for k in resultados:
        plt.figure(figsize=(10, 6))

        alfa_c = pegar_alfa_critico(resultados, k) #chamada da função para encontrar α_c

        for n in resultados[k]:
            plt.plot(resultados[k][n]['alfas'],
                     resultados[k][n]['probabilidades'],
                     marker='o',
                     label=f'n={n}')
        plt.axvline(x=alfa_c,
                    color='black',
                    linestyle='-',
                    label=f'α_c = {alfa_c:.2f}') #linha que mostra o α_c no gráfico

        plt.title(f'Transição de Fase {k}-SAT')
        plt.xlabel('α')
        plt.ylabel('Probabilidade de Satisfazibilidade')
        plt.legend()
        plt.grid()
        plt.savefig(f'{k}-SAT_probabilidade.png')
        plt.close()

    # Gráfico de tempo
    for k in resultados:
        alfa_c = pegar_alfa_critico(resultados, k)
        plt.figure(figsize=(10, 6))
        for n in resultados[k]:
            plt.plot(resultados[k][n]['alfas'],
                     resultados[k][n]['tempos'],
                     marker='o',
                     label=f'n={n}')
        plt.axvline(x=alfa_c,
                    color='black',
                    linestyle='-',
                    label=f'α_c = {alfa_c:.2f}')

        plt.title(f'Tempo Médio {k}-SAT')
        plt.xlabel('α')
        plt.ylabel('Tempo (s)')
        plt.legend()
        plt.grid()
        plt.savefig(f'{k}-SAT_tempo.png')
        plt.close()

In [22]:
def pegar_alfa_critico(resultados, k):

    #encontra α_c com base no pico máximo de tempo que o solver demora para dar uma resposta

    tempos_globais = []
    alfas_globais = []

    for n in resultados[k]:
        tempos_globais.extend(resultados[k][n]['tempos'])
        alfas_globais.extend(resultados[k][n]['alfas'])


    pico_tempo_index = np.argmax(tempos_globais)
    alfa_critico_global = alfas_globais[pico_tempo_index]

    return alfa_critico_global

In [27]:
resultados = executar_experimento()



Processando 3-SAT com n=50...
α=1.0 | Prob: 1.00 | Tempo: 0.0000s
α=1.1 | Prob: 1.00 | Tempo: 0.0000s
α=1.2 | Prob: 1.00 | Tempo: 0.0000s
α=1.3 | Prob: 1.00 | Tempo: 0.0000s
α=1.4 | Prob: 1.00 | Tempo: 0.0000s
α=1.5 | Prob: 1.00 | Tempo: 0.0000s
α=1.6 | Prob: 1.00 | Tempo: 0.0000s
α=1.7 | Prob: 1.00 | Tempo: 0.0000s
α=1.8 | Prob: 1.00 | Tempo: 0.0000s
α=1.9 | Prob: 1.00 | Tempo: 0.0000s
α=2.0 | Prob: 1.00 | Tempo: 0.0000s
α=2.1 | Prob: 1.00 | Tempo: 0.0000s
α=2.2 | Prob: 1.00 | Tempo: 0.0000s
α=2.3 | Prob: 1.00 | Tempo: 0.0000s
α=2.4 | Prob: 1.00 | Tempo: 0.0000s
α=2.5 | Prob: 1.00 | Tempo: 0.0000s
α=2.6 | Prob: 1.00 | Tempo: 0.0000s
α=2.7 | Prob: 1.00 | Tempo: 0.0000s
α=2.8 | Prob: 1.00 | Tempo: 0.0000s
α=2.9 | Prob: 1.00 | Tempo: 0.0000s
α=3.0 | Prob: 1.00 | Tempo: 0.0000s
α=3.1 | Prob: 1.00 | Tempo: 0.0000s
α=3.2 | Prob: 1.00 | Tempo: 0.0000s
α=3.3 | Prob: 1.00 | Tempo: 0.0000s
α=3.4 | Prob: 1.00 | Tempo: 0.0000s
α=3.5 | Prob: 1.00 | Tempo: 0.0001s
α=3.6 | Prob: 1.00 | Tempo: 0.000

In [30]:
gerar_graficos(resultados)


In [10]:
print(resultados)

{3: {50: {'alfas': [1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0, 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5.0, 5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6.0, 6.1, 6.2, 6.3, 6.4, 6.5, 6.6, 6.7, 6.8, 6.9, 7.0, 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8, 7.9, 8.0, 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9.0, 9.1, 9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9, 10.0], 'probabilidades': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.85, 0.9, 0.7, 0.55, 0.5, 0.35, 0.3, 0.1, 0.2, 0.15, 0.1, 0.05, 0.1, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'tempos': [1.8033249999960256e-05, 1.75653